In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/05/29 10:36:48 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/harsha/notebooks/mlops-learning/02-experiment-tracking/mlruns/1', creation_time=1685356608775, experiment_id='1', last_update_time=1685356608775, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [31]:
df = pd.read_parquet('/home/harsha/notebooks/data/green_tripdata_2021-03.parquet')
df.to_csv('/home/harsha/notebooks/data/green_tripdata_2021-03.csv')

In [7]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('/home/harsha/notebooks/data/green_tripdata_2021-01.csv')
df_val = read_dataframe('/home/harsha/notebooks/data/green_tripdata_2021-02.csv')

/tmp/ipykernel_26741/3970424726.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [15]:
len(df_train), len(df_val)

(73908, 61921)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [17]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715199477344

In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
with mlflow.start_run():

    mlflow.set_tag("developer", "harsha")

    mlflow.log_param("train-data-path", "/home/harsha/notebooks/data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "/home/harsha/notebooks/data/green_tripdata_2021-02.csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

#     mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [23]:
import xgboost as xgb

In [24]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [25]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [26]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [27]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:55:16] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.  
[0]	validation-rmse:13.48707                                                                                            
[1]	validation-rmse:9.72174                                                                                             
[2]	validation-rmse:8.03290                                                                                             
[3]	validation-rmse:7.31563                                                                                             
[4]	validation-rmse:7.00766                                                                                             
[5]	validation-rmse:6.86633                                                                                             
[6]	validation-rmse:6.79710                                                                                             
[7]	validation-rmse:6.75411     

[133]	validation-rmse:6.67736                                                                                           
[134]	validation-rmse:6.67729                                                                                           
[135]	validation-rmse:6.67736                                                                                           
[136]	validation-rmse:6.67740                                                                                           
[137]	validation-rmse:6.67703                                                                                           
[138]	validation-rmse:6.67684                                                                                           
[139]	validation-rmse:6.67673                                                                                           
[140]	validation-rmse:6.67693                                                                                           
[141]	validation-rmse:6.67695   

[120]	validation-rmse:6.44296                                                                                           
[121]	validation-rmse:6.44250                                                                                           
[122]	validation-rmse:6.44220                                                                                           
[123]	validation-rmse:6.44150                                                                                           
[124]	validation-rmse:6.44098                                                                                           
[125]	validation-rmse:6.44043                                                                                           
[126]	validation-rmse:6.44011                                                                                           
[127]	validation-rmse:6.43973                                                                                           
[128]	validation-rmse:6.43946   

[254]	validation-rmse:6.40643                                                                                           
[255]	validation-rmse:6.40625                                                                                           
[256]	validation-rmse:6.40605                                                                                           
[257]	validation-rmse:6.40578                                                                                           
[258]	validation-rmse:6.40560                                                                                           
[259]	validation-rmse:6.40549                                                                                           
[260]	validation-rmse:6.40544                                                                                           
[261]	validation-rmse:6.40526                                                                                           
[262]	validation-rmse:6.40513   

[388]	validation-rmse:6.39068                                                                                           
[389]	validation-rmse:6.39071                                                                                           
[390]	validation-rmse:6.39055                                                                                           
[391]	validation-rmse:6.39034                                                                                           
[392]	validation-rmse:6.39013                                                                                           
[393]	validation-rmse:6.38997                                                                                           
[394]	validation-rmse:6.38996                                                                                           
[395]	validation-rmse:6.38986                                                                                           
[396]	validation-rmse:6.38979   

...

[111]	validation-rmse:6.56056                                                                                           
[112]	validation-rmse:6.56019                                                                                           
[113]	validation-rmse:6.55996                                                                                           
[114]	validation-rmse:6.55985                                                                                           
[115]	validation-rmse:6.55966                                                                                           
[116]	validation-rmse:6.55902                                                                                           
[117]	validation-rmse:6.55863                                                                                           
[118]	validation-rmse:6.55845                                                                                           
[119]	validation-rmse:6.55823   

[245]	validation-rmse:6.53446                                                                                           
[246]	validation-rmse:6.53451                                                                                           
[247]	validation-rmse:6.53459                                                                                           
[248]	validation-rmse:6.53443                                                                                           
[249]	validation-rmse:6.53432                                                                                           
[250]	validation-rmse:6.53415                                                                                           
[251]	validation-rmse:6.53421                                                                                           
[252]	validation-rmse:6.53416                                                                                           
[253]	validation-rmse:6.53405   

[379]	validation-rmse:6.52606                                                                                           
[380]	validation-rmse:6.52601                                                                                           
[381]	validation-rmse:6.52578                                                                                           
[382]	validation-rmse:6.52598                                                                                           
[383]	validation-rmse:6.52577                                                                                           
[384]	validation-rmse:6.52594                                                                                           
[385]	validation-rmse:6.52589                                                                                           
[386]	validation-rmse:6.52581                                                                                           
[387]	validation-rmse:6.52579   

[513]	validation-rmse:6.52578                                                                                           
[514]	validation-rmse:6.52568                                                                                           
[515]	validation-rmse:6.52562                                                                                           
[516]	validation-rmse:6.52558                                                                                           
[517]	validation-rmse:6.52554                                                                                           
[518]	validation-rmse:6.52559                                                                                           
[519]	validation-rmse:6.52560                                                                                           
[520]	validation-rmse:6.52552                                                                                           
[521]	validation-rmse:6.52545   

[101]	validation-rmse:6.51243                                                                                           
[102]	validation-rmse:6.51211                                                                                           
[103]	validation-rmse:6.51187                                                                                           
[104]	validation-rmse:6.51152                                                                                           
[105]	validation-rmse:6.51102                                                                                           
[106]	validation-rmse:6.51050                                                                                           
[107]	validation-rmse:6.51001                                                                                           
[108]	validation-rmse:6.50991                                                                                           
[109]	validation-rmse:6.50957   

In [28]:
mlflow.xgboost.autolog(disable=True)

In [29]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.193414970818923,
        'max_depth': 17,
        'min_child_weight': 1.0285910725263858,
        'objective': 'reg:linear',
        'reg_alpha': 0.0118726814260229,
        'reg_lambda': 0.20626693919323963,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [30]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/home/harsha/notebooks/data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "/home/harsha/notebooks/data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/harsha/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
